# Create CSV

In [1]:
# change path according to your local path
# the path should be set to the folder containing the 60 gz files
dset_path = "D:\\_coding\\_wd\\_python\\uni\\Behavioral-Context-Recognition\\dataset\\unzipped_dataset"

In [2]:
# read the all the csv files
import os
import gzip
import pandas as pd

df_list=[]
for gzFile in os.listdir(dset_path):
   path_gz = dset_path + "\\" + gzFile
   with gzip.open(path_gz) as f:
      df = pd.read_csv(f, header=0)
      df.insert(0,'uuid',gzFile[:-23])
      df_list.append(df)

df_dset = pd.concat(df_list, axis=0, ignore_index=True)
df_dset.head()

,uuid,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,...,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,00EABED2-271D-49D8-B599-1D4A09240601,1444079161,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
1,00EABED2-271D-49D8-B599-1D4A09240601,1444079221,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
2,00EABED2-271D-49D8-B599-1D4A09240601,1444079281,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
3,00EABED2-271D-49D8-B599-1D4A09240601,1444079341,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
4,00EABED2-271D-49D8-B599-1D4A09240601,1444079431,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2


In [3]:
# save csv as it is in directory:
dset_final_path = "D:\\_coding\\_wd\\_python\\uni\\Behavioral-Context-Recognition\\dataset"

df_dset.to_csv(dset_final_path+"\\complete_dataset.csv", index=False)

## Data Preparation
Following are a set of operation to clean and create the dataset csv for model learning.

First we import the complete and unaltered dataset:

In [1]:
import pandas as pd

# read the complete dataset csv:
dset_path = "D:\\_coding\\_wd\\_python\\uni\\Behavioral-Context-Recognition\\dataset\\complete_dataset.csv"
df_dset = pd.read_csv(dset_path)

### WALKING :-
In this section we'll create all the datasets for WALKING/NON WALKING classification. For this purpose, we'll select
the WALKING label and only those features we analyzed in EDA notebook and found they were useful for this classification.


In [2]:
df_dset_copy = df_dset.copy()

columns = df_dset_copy.columns

list_columns_to_delete = []
for col in columns:
    if any(word in col for word in ["gyro","watch_acc","raw_acc"]):
        continue
    elif "label:FIX_walking" in col:
        continue
    else:
        list_columns_to_delete.append(col)

# delete timestamps too because the series are already ordered
df_dset_copy.drop(columns=list_columns_to_delete, inplace=True)
print("Columns deleted.")

df_dset_walking_3f = df_dset_copy

Columns deleted.


In [3]:
df_dset_walking_3f.columns.values


array(['raw_acc:magnitude_stats:mean', 'raw_acc:magnitude_stats:std',
       'raw_acc:magnitude_stats:moment3',
       'raw_acc:magnitude_stats:moment4',
       'raw_acc:magnitude_stats:percentile25',
       'raw_acc:magnitude_stats:percentile50',
       'raw_acc:magnitude_stats:percentile75',
       'raw_acc:magnitude_stats:value_entropy',
       'raw_acc:magnitude_stats:time_entropy',
       'raw_acc:magnitude_spectrum:log_energy_band0',
       'raw_acc:magnitude_spectrum:log_energy_band1',
       'raw_acc:magnitude_spectrum:log_energy_band2',
       'raw_acc:magnitude_spectrum:log_energy_band3',
       'raw_acc:magnitude_spectrum:log_energy_band4',
       'raw_acc:magnitude_spectrum:spectral_entropy',
       'raw_acc:magnitude_autocorrelation:period',
       'raw_acc:magnitude_autocorrelation:normalized_ac',
       'raw_acc:3d:mean_x', 'raw_acc:3d:mean_y', 'raw_acc:3d:mean_z',
       'raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'raw_acc:3d:std_z',
       'raw_acc:3d:ro_xy', 'raw_acc:3d:r

#### Data Cleaning
At first, we shall see if data is already cleaned or if further preprocessing is required. We already assessed in EDA
notebook that some columns contain <code>nan</code> values. So we'll start by cleaning up those values for features columns.

In [4]:

print(df_dset_walking_3f.loc[:,'raw_acc:magnitude_stats:mean':'raw_acc:3d:ro_yz'].isna().sum())
print(df_dset_walking_3f.loc[:,'proc_gyro:magnitude_stats:mean':'proc_gyro:3d:ro_yz'].isna().sum())
print(df_dset_walking_3f.loc[:,'watch_acceleration:magnitude_stats:mean':'watch_acceleration:relative_directions:avr_cosine_similarity_lag_range4'].isna().sum())

raw_acc:magnitude_stats:mean                       290
raw_acc:magnitude_stats:std                        290
raw_acc:magnitude_stats:moment3                    290
raw_acc:magnitude_stats:moment4                    290
raw_acc:magnitude_stats:percentile25               290
raw_acc:magnitude_stats:percentile50               290
raw_acc:magnitude_stats:percentile75               290
raw_acc:magnitude_stats:value_entropy              290
raw_acc:magnitude_stats:time_entropy               290
raw_acc:magnitude_spectrum:log_energy_band0        290
raw_acc:magnitude_spectrum:log_energy_band1        290
raw_acc:magnitude_spectrum:log_energy_band2        290
raw_acc:magnitude_spectrum:log_energy_band3        290
raw_acc:magnitude_spectrum:log_energy_band4        290
raw_acc:magnitude_spectrum:spectral_entropy        290
raw_acc:magnitude_autocorrelation:period           290
raw_acc:magnitude_autocorrelation:normalized_ac    290
raw_acc:3d:mean_x                                  290
raw_acc:3d

In [5]:
features_names = df_dset_walking_3f.columns.values[0:-1].tolist()

print("Missing values in features columns (before) = ", df_dset_walking_3f.loc[:,features_names].isna().sum().sum())
df_dset_walking_3f.dropna(subset=features_names, inplace=True)
print("Missing values in features columns (after) = ", df_dset_walking_3f.loc[:,features_names].isna().sum().sum())

Missing values in features columns (before) =  6563247
Missing values in features columns (after) =  0


Now for label column:

In [6]:
# set all nan values of label:FIX_walking to 0
print("Number of NaN values (before): ", df_dset_walking_3f.loc[df_dset_walking_3f["label:FIX_walking"].isna(), ["label:FIX_walking"]].shape[0])
df_dset_walking_3f.loc[df_dset_walking_3f["label:FIX_walking"].isna(), ["label:FIX_walking"]] = 0.0
print("Number of NaN values (after): ", df_dset_walking_3f.loc[df_dset_walking_3f["label:FIX_walking"].isna(), ["label:FIX_walking"]].shape[0])

Number of NaN values (before):  34718
Number of NaN values (after):  0


In [9]:
df_dset_walking_3f.reset_index(inplace=True)
df_dset_walking_3f

,index,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,watch_acceleration:spectrum:z_log_energy_band1,watch_acceleration:spectrum:z_log_energy_band2,watch_acceleration:spectrum:z_log_energy_band3,watch_acceleration:spectrum:z_log_energy_band4,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range0,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range1,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range2,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range3,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range4,label:FIX_walking
0,0,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,6.684605,...,0.004994,0.023702,0.022539,0.083756,0.999849,0.999835,0.999836,0.999832,0.999819,0.0
1,1,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,6.684603,...,0.003815,0.025582,0.023285,0.129195,0.999835,0.999827,0.999817,0.999820,0.999828,0.0
2,2,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,6.684605,...,0.007040,0.020717,0.026889,0.080554,0.999817,0.999797,0.999790,0.999791,0.999824,0.0
3,3,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,6.684605,...,0.004311,0.034215,0.035575,0.090120,0.999854,0.999845,0.999842,0.999841,0.999829,0.0
4,4,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,6.683904,...,1.742936,1.477211,1.302678,2.700446,0.986681,0.962466,0.955313,0.944759,0.995185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232868,366284,1.044478,0.001459,-0.000447,0.001928,1.043584,1.044439,1.045451,2.557106,6.684611,...,2.793748,3.069341,1.653165,0.000000,0.975794,0.939543,0.818400,0.755940,0.778336,0.0
232869,366285,1.045003,0.001521,0.000802,0.002004,1.043932,1.044956,1.045980,2.510352,6.684611,...,3.839144,4.369760,1.329904,0.000000,0.888645,0.776227,0.353599,0.325917,-0.103433,0.0
232870,366286,1.044018,0.001528,-0.000719,0.002038,1.043013,1.044054,1.045033,2.587285,6.684611,...,4.570503,4.411898,3.227623,0.000000,0.901224,0.768997,0.431188,0.305545,0.579282,0.0
232871,366287,1.043469,0.048007,-0.070631,0.147004,1.041401,1.043133,1.044632,0.542064,6.683448,...,3.424182,3.070862,2.806717,0.000000,0.958302,0.895361,0.726545,0.337854,-0.262308,0.0


In [11]:
df_dset_walking_3f.drop(axis=1, columns=["index"], inplace=True)
df_dset_walking_3f

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,watch_acceleration:spectrum:z_log_energy_band1,watch_acceleration:spectrum:z_log_energy_band2,watch_acceleration:spectrum:z_log_energy_band3,watch_acceleration:spectrum:z_log_energy_band4,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range0,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range1,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range2,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range3,watch_acceleration:relative_directions:avr_cosine_similarity_lag_range4,label:FIX_walking
0,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,6.684605,5.043970,...,0.004994,0.023702,0.022539,0.083756,0.999849,0.999835,0.999836,0.999832,0.999819,0.0
1,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,6.684603,5.043367,...,0.003815,0.025582,0.023285,0.129195,0.999835,0.999827,0.999817,0.999820,0.999828,0.0
2,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,6.684605,5.043599,...,0.007040,0.020717,0.026889,0.080554,0.999817,0.999797,0.999790,0.999791,0.999824,0.0
3,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,6.684605,5.043263,...,0.004311,0.034215,0.035575,0.090120,0.999854,0.999845,0.999842,0.999841,0.999829,0.0
4,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,6.683904,5.042779,...,1.742936,1.477211,1.302678,2.700446,0.986681,0.962466,0.955313,0.944759,0.995185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232868,1.044478,0.001459,-0.000447,0.001928,1.043584,1.044439,1.045451,2.557106,6.684611,5.043712,...,2.793748,3.069341,1.653165,0.000000,0.975794,0.939543,0.818400,0.755940,0.778336,0.0
232869,1.045003,0.001521,0.000802,0.002004,1.043932,1.044956,1.045980,2.510352,6.684611,5.043931,...,3.839144,4.369760,1.329904,0.000000,0.888645,0.776227,0.353599,0.325917,-0.103433,0.0
232870,1.044018,0.001528,-0.000719,0.002038,1.043013,1.044054,1.045033,2.587285,6.684611,5.043217,...,4.570503,4.411898,3.227623,0.000000,0.901224,0.768997,0.431188,0.305545,0.579282,0.0
232871,1.043469,0.048007,-0.070631,0.147004,1.041401,1.043133,1.044632,0.542064,6.683448,5.043480,...,3.424182,3.070862,2.806717,0.000000,0.958302,0.895361,0.726545,0.337854,-0.262308,0.0


#### Create CSV for walking label

In [12]:
# save csv for walking with all the 3 features:
dset_final_path = "D:\\_coding\\_wd\\_python\\uni\\Behavioral-Context-Recognition\\dataset"
df_dset_walking_3f.to_csv(dset_final_path+"\\dataset_walking_3f.csv", index=False)
print("Dataset saved to csv.")

Dataset saved to csv.


In [14]:
# save csv for walking with all gyro features:
dset_final_path = "D:\\_coding\\_wd\\_python\\uni\\Behavioral-Context-Recognition\\dataset"

df_dset_walking_3f_copy = df_dset_walking_3f.copy()

columns = df_dset_walking_3f_copy.columns
list_columns_to_delete = []

for col in columns:
    if any(word in col for word in ["gyro","label"]):
        continue
    else:
        list_columns_to_delete.append(col)

df_dset_walking_3f_copy.drop(columns=list_columns_to_delete, inplace=True)
print("Columns deleted.")

df_dset_walking_3f_copy.to_csv(dset_final_path+"\\dataset_walking_gyro_allfeatures.csv", index=False)
print("Dataset saved to csv.")

Columns deleted.
Dataset saved to csv.
